# 📈 05 — Model Evaluation

We evaluate:
- Final model (XGBoost)
- ROC curve
- PR curve
- SHAP values
- Threshold selection

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve

import shap

from src.data_prep import load_raw_data, train_val_test_split, build_preprocessor
from src.models import get_xgb_model
from sklearn.pipeline import Pipeline

In [ ]:
df = load_raw_data()
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df)

pre = build_preprocessor()
model = get_xgb_model()

pipe = Pipeline([("preprocessor", pre), ("model", model)])
pipe.fit(X_train, y_train)

y_val_proba = pipe.predict_proba(X_val)[:, 1]

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_val, y_val_proba)
plt.plot(recall, precision)
plt.title("Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.show()

In [ ]:
xgb = pipe.named_steps["model"]
preprocessed_val = pipe.named_steps["preprocessor"].transform(X_val)

explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(preprocessed_val)

shap.summary_plot(shap_values, preprocessed_val)

# Evaluation Summary

- XGBoost outperforms baseline and RF.
- SHAP shows which PCA components matter most.
- Threshold must be tuned to satisfy business constraints.